In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split

In [2]:
!pip install -e ..

Obtaining file:///home/daniel/code/llenci/RealEstateTracker
  Preparing metadata (setup.py) ... done


  Attempting uninstall: RealEstateTracker
    Found existing installation: RealEstateTracker 1.0
    Uninstalling RealEstateTracker-1.0:
      Successfully uninstalled RealEstateTracker-1.0
  Running setup.py develop for RealEstateTracker


In [3]:
from RealEstateTracker import preprocessing, utils


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
df = preprocessing.preprocess()


/home/daniel/code/llenci/RealEstateTracker/RealEstateTracker/preprocessing.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonull["surface"] = new[0]
/home/daniel/code/llenci/RealEstateTracker/RealEstateTracker/preprocessing.py:44: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  new[0] = new[0].str.replace(".","")
/home/daniel/code/llenci/RealEstateTracker/RealEstateTracker/preprocessing.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [6]:
X,y = preprocessing.X_y(df)

In [7]:
X_train, X_test, y_train, y_test = preprocessing.split(X,y, test_size = 0.2, takelog=True)


DEBUG: y:
          price
0     13.247232
1     12.645939
2     12.301383
3     12.049419
4     12.818552
...         ...
5748  13.151922
5749  12.542545
5750  12.112311
5751  12.301383
5752  12.278393

[5753 rows x 1 columns]


In [8]:
import inspect

lines = inspect.getsource(preprocessing.split)
print(lines)

def split(X, y, test_size = 0.2, takelog=True):
    if takelog:
        y = np.log(y)
    print(f"DEBUG: y:\n{y}")
    """Splits data AND returns the log scale of y"""
    return train_test_split(X, y, test_size=test_size)



In [9]:
y_test

,price
2803,12.487485
64,12.676076
1223,13.527828
570,11.976659
741,12.429216
...,...
2782,11.775290
2480,13.721200
1950,13.455974
3509,12.296827


In [17]:
model = preprocessing.model_basic

In [18]:
model.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('robustscaler',
                                                  RobustScaler(),
                                                  ['rooms', 'surface']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['neighborhood',
                                                   'nhousetype'])])),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              co...
                              gamma=0, gpu_id=-1, importance_type=None,
                              interaction_constra

In [19]:
y_pred = utils.custom_pred(model, X_test)

In [20]:
y_pred

array([271284.5 , 304970.62, 795349.06, ..., 700558.3 , 196276.52,
       482346.1 ], dtype=float32)

In [14]:
from sklearn.metrics import mean_squared_log_error

In [23]:
y_test = np.exp(y_test)

In [24]:
mean_squared_log_error(y_test , y_pred)

0.0713299291765771

In [26]:
utils.save_model(model)

Model saved as model.joblib


In [16]:
y_pred

array([271284.5 , 304970.62, 795349.06, ..., 700558.3 , 196276.52,
       482346.1 ], dtype=float32)